In [1]:
import cv2
import h5py

import tensorflow as tf
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

from glob import glob 
from tqdm.notebook import tqdm 

In [26]:
tf.random.set_seed(
    1234
)

In [27]:
train_data = h5py.File('./data/camelyonpatch_level_2_split_train_x.h5','r+')    
train_data = list(train_data['x'])
np.shape(train_data)

(262144, 96, 96, 3)

In [28]:
train_label = h5py.File('./data/camelyonpatch_level_2_split_train_y.h5','r+')    
train_label = list(train_label['y'])
np.shape(train_label)

(262144, 1, 1, 1)

In [29]:
train_label_new = []

for i in train_label:
    train_label_new.append(i[0][0][0])

np.shape(train_label_new)

(262144,)

In [30]:
model = tf.keras.applications.efficientnet_v2.EfficientNetV2L(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=[96, 96, 3],
    pooling=None,
    classes=2,
    classifier_activation='softmax',
    include_preprocessing=True
)

In [31]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [32]:
tf.keras.backend.clear_session

<function keras.backend.clear_session()>

In [33]:
model.fit(
    x = np.array(train_data),
    y = np.array(train_label_new),
    validation_split = 0.2,
    epochs = 10,
    shuffle = True,
    batch_size = 8,
)

Epoch 1/10
 4905/26215 [====>.........................] - ETA: 2:01:47 - loss: 1.0157 - accuracy: 0.5906

In [ ]:
model.save_weights('./save/model_10_epoch.h5')

### Inf

In [8]:
model.load_weights('./save/model.h5')

In [9]:
test_file_path = glob('../pcam_data/test_data/*')
len(test_file_path)

250

In [10]:
img = cv2.imread('../pcam_data/test_data/BC_01_0011/0.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = tf.expand_dims(img, axis=0)
model.predict(np.array(img))
# np.argmax(model.predict(np.array(img)))

1/1 [==============================] - 9s 9s/step


array([[0.99031305, 0.00968695]], dtype=float32)

In [11]:
test_file_path = glob('../pcam_data/train_data/*')
len(test_file_path)

1000

In [23]:
submission_label = []

for i in tqdm(test_file_path[0:1]):
    tmp_label = []
    test_image_path = glob(i + '/*.*')

    for j in test_image_path:
        img = cv2.imread(j)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = tf.expand_dims(img, axis=0)
        perdiction = model.predict(np.array(img))
        loc = np.argmax(perdiction)
        if loc == 1:
            if perdiction[0][loc] > 0.9:
                tmp_label.append(1)
            else:
                tmp_label.append(0)
        else:
            tmp_label.append(0)

    if np.max(tmp_label) == 1:
        submission_label.append(1)
    else:
        submission_label.append(0)

  0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 75ms/step


In [25]:
i

'../pcam_data/train_data\\BC_01_0001'

In [24]:
submission_label

[1]